In [1]:
import pyspark
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

df = spark.read.parquet('data/yellow_tripdata_2024-10.parquet')
df = df.repartition(4)
df.write.parquet('data/pq/')

In [8]:
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [7]:
df.registerTempTable('trips_data')  

spark.sql("""

SELECT count(1) FROM trips_data 
WHERE DATE(tpep_pickup_datetime) = '2024-10-15' AND DATE(tpep_dropoff_datetime) = '2024-10-15';
""").show()

+--------+
|count(1)|
+--------+
|  127993|
+--------+



In [9]:
spark.sql("""
SELECT 
    (unix_timestamp(tpep_dropoff_datetime) - unix_timestamp(tpep_pickup_datetime)) / 3600 AS trip_duration
FROM trips_data
ORDER BY trip_duration DESC
LIMIT 1;
""").show() 

+------------------+
|     trip_duration|
+------------------+
|162.61777777777777|
+------------------+



In [11]:

lookup_df = spark.read.option("header", "true").csv('taxi_zone_lookup.csv')

lookup_df.registerTempTable('lookup')  

spark.sql("""

SELECT lookup.Zone , count(1) as num_trips FROM trips_data 
INNER JOIN lookup ON lookup.LocationID = trips_data.PULocationID
GROUP BY lookup.Zone
ORDER BY num_trips ASC;
""").show()  


+--------------------+---------+
|                Zone|num_trips|
+--------------------+---------+
|Governor's Island...|        1|
|       Rikers Island|        2|
|       Arden Heights|        2|
|         Jamaica Bay|        3|
| Green-Wood Cemetery|        3|
|Charleston/Totten...|        4|
|Eltingville/Annad...|        4|
|   Rossville/Woodrow|        4|
|       Port Richmond|        4|
|       West Brighton|        4|
|        Crotona Park|        6|
|         Great Kills|        6|
|     Mariners Harbor|        7|
|Heartland Village...|        7|
|Saint George/New ...|        9|
|             Oakwood|        9|
|       Broad Channel|       10|
|New Dorp/Midland ...|       10|
|         Westerleigh|       12|
|     Pelham Bay Park|       12|
+--------------------+---------+
only showing top 20 rows

